# Setup

In [1]:
import json

import pandas as pd
from google.cloud import securitycenter_v1
from tqdm import tqdm

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", None)

# Functions

In [2]:
scc_client = securitycenter_v1.SecurityCenterClient()


def get_finding_configs():
    with open("finding_configs.json", "r") as fp:
        return json.load(fp)


def get_findings(org_id):
    resp = scc_client.list_findings(
        securitycenter_v1.ListFindingsRequest(
            parent=f"organizations/{org_id}/sources/-",
            page_size=1000,
        )
    )
    return tqdm(resp, total=resp.total_size, unit="findings", desc="Loading Findings")


def get_assets(org_id):
    resp = scc_client.list_assets(
        securitycenter_v1.ListAssetsRequest(
            parent=f"organizations/{org_id}",
            page_size=1000,
        )
    )
    return tqdm(resp, total=resp.total_size, unit="assets", desc="Loading Assets")


def compute_verdicts(assets, findings, finding_configs):
    susceptible_category_map = {}
    for finding_config in finding_configs:
        for resource_type in finding_config["resource_types"]:
            susceptible_category_map.setdefault(resource_type, []).append(
                finding_config["finding_type"]
            )

    finding_type_compliance_map = {}
    for finding_config in finding_configs:
        for compliance in finding_config["compliance_metadata"]:
            finding_type_compliance_map.setdefault(
                finding_config["finding_type"], []
            ).append(compliance)

    vulnerabilities = {
        (f.finding.resource_name, f.finding.category)
        for f in findings
        if f.finding.state == securitycenter_v1.Finding.State.ACTIVE
    }

    return pd.DataFrame(
        [
            {
                "resource_name": asset.asset.security_center_properties.resource_name,
                "resource_type": asset.asset.security_center_properties.resource_type,
                "finding_type": finding_type,
                "verdict": (
                    "positive"
                    if (
                        asset.asset.security_center_properties.resource_name,
                        finding_type,
                    )
                    in vulnerabilities
                    else "negative"
                ),
                "project": asset.asset.security_center_properties.resource_project,
                "benchmarks": finding_type_compliance_map.get(finding_type, []),
            }
            for asset in assets
            for finding_type in susceptible_category_map.get(
                asset.asset.security_center_properties.resource_type, []
            )
        ]
    )

# Load Findings and Assets

In [3]:
org_id = "129624834409"
findings = list(get_findings(org_id))
assets = list(get_assets(org_id))

Loading Assets: 100%|██████████| 6794/6794 [00:17<00:00, 387.63assets/s]


In [4]:
finding_configs = get_finding_configs()
vdf = compute_verdicts(assets, findings, finding_configs)

# Verdicts

In [5]:
vdf

,resource_name,resource_type,finding_type,verdict,project,benchmarks
0,//cloudresourcemanager.googleapis.com/organizations/129624834409,google.cloud.resourcemanager.Organization,ADMIN_SERVICE_ACCOUNT,positive,,"[CIS 1.0 Level 1, CIS 1.1 Level 1]"
1,//cloudresourcemanager.googleapis.com/organizations/129624834409,google.cloud.resourcemanager.Organization,AUDIT_LOGGING_DISABLED,positive,,"[CIS 1.0 Level 1, CIS 1.1 Level 1, PCI, NIST, ISO]"
2,//cloudresourcemanager.googleapis.com/organizations/129624834409,google.cloud.resourcemanager.Organization,KMS_ROLE_SEPARATION,positive,,"[CIS 1.0 Level 2, CIS 1.1 Level 2, NIST, ISO]"
3,//cloudresourcemanager.googleapis.com/organizations/129624834409,google.cloud.resourcemanager.Organization,MFA_NOT_ENFORCED,positive,,"[CIS 1.0 Level 1, CIS 1.1 Level 1, PCI, NIST, ISO]"
4,//cloudresourcemanager.googleapis.com/organizations/129624834409,google.cloud.resourcemanager.Organization,NON_ORG_IAM_MEMBER,positive,,"[CIS 1.0 Level 1, CIS 1.1 Level 1, PCI, NIST, ISO]"
...,...,...,...,...,...,...
17959,//compute.googleapis.com/projects/kleon-sandbox/regions/us-east1/backendServices/7021574685558114040,google.compute.RegionBackendService,ORG_POLICY_LOCATION_RESTRICTION,negative,//cloudresourcemanager.googleapis.com/projects/335767247403,[]
17960,//compute.googleapis.com/projects/weak-policy-243021/regions/us-west1/backendServices/3311081193619846270,google.compute.RegionBackendService,ORG_POLICY_LOCATION_RESTRICTION,negative,//cloudresourcemanager.googleapis.com/projects/93853049618,[]
17961,//compute.googleapis.com/projects/csek-243518/regions/us-central1/disks/3365701890509221548,google.compute.RegionDisk,ORG_POLICY_LOCATION_RESTRICTION,negative,//cloudresourcemanager.googleapis.com/projects/1037068353870,[]
17962,//dataproc.googleapis.com/projects/csek-243518/regions/us-central1/clusters/cluster-f95b,google.dataproc.Cluster,ORG_POLICY_LOCATION_RESTRICTION,negative,//cloudresourcemanager.googleapis.com/projects/1037068353870,[]


# Compliance Coverage

## By Benchmark

In [6]:
benchmarks = {
    benchmark
    for finding_config in finding_configs
    for benchmark in finding_config["compliance_metadata"]
}
benchmarks

{'CIS 1.0 Level 1',
 'CIS 1.0 Level 2',
 'CIS 1.1 Level 1',
 'CIS 1.1 Level 2',
 'ISO',
 'NIST',
 'PCI'}

In [7]:
rows = []
for benchmark in benchmarks:
    tdf = vdf[vdf["benchmarks"].apply(lambda b: benchmark in b)]
    rows.append(
        {
            "benchmark": benchmark,
            "positive": len(tdf[tdf["verdict"] == "positive"]),
            "total": len(tdf),
        }
    )
tdf = pd.DataFrame(rows)
tdf = tdf.assign(vuln_percent=tdf["positive"] * 100 / tdf["total"])
tdf = tdf.set_index("benchmark").sort_index()
tdf

,positive,total,vuln_percent
benchmark,,,
CIS 1.0 Level 1,1197,2463,48.599269
CIS 1.0 Level 2,650,3181,20.433826
CIS 1.1 Level 1,1088,2590,42.007722
CIS 1.1 Level 2,208,1462,14.227086
ISO,1657,11402,14.532538
NIST,1665,11433,14.563107
PCI,2065,12020,17.179700


## Filtered by a Project

In [10]:
rows = []
for benchmark in benchmarks:
    tdf = vdf[
        (vdf["benchmarks"].apply(lambda b: benchmark in b))
        & (
            vdf["project"]
            == "//cloudresourcemanager.googleapis.com/projects/1037068353870"
        )
    ]
    rows.append(
        {
            "benchmark": benchmark,
            "positive": len(tdf[tdf["verdict"] == "positive"]),
            "total": len(tdf),
        }
    )
tdf = pd.DataFrame(rows)
tdf = tdf.assign(vuln_percent=tdf["positive"] * 100 / tdf["total"])
tdf = tdf.set_index("benchmark").sort_index()
tdf

,positive,total,vuln_percent
benchmark,,,
CIS 1.0 Level 1,52,90,57.777778
CIS 1.0 Level 2,30,45,66.666667
CIS 1.1 Level 1,51,100,51.000000
CIS 1.1 Level 2,11,25,44.000000
ISO,44,155,28.387097
NIST,39,149,26.174497
PCI,48,160,30.000000


## By Finding Type

In [11]:
tdf = vdf[["finding_type", "verdict"]].pivot_table(
    index="finding_type", columns="verdict", aggfunc="size", fill_value=0
)
tdf = tdf.assign(total=tdf.sum(axis="columns"))
tdf = tdf.drop(columns=["negative"])
tdf = tdf.assign(vuln_percent=tdf["positive"] * 100 / tdf["total"])
tdf

verdict,positive,total,vuln_percent
finding_type,,,
ADMIN_SERVICE_ACCOUNT,13,54,24.074074
API_KEY_APIS_UNRESTRICTED,7,41,17.073171
API_KEY_APPS_UNRESTRICTED,3,41,7.317073
API_KEY_EXISTS,7,41,17.073171
API_KEY_NOT_ROTATED,7,41,17.073171
AUDIT_CONFIG_NOT_MONITORED,38,41,92.682927
AUDIT_LOGGING_DISABLED,37,42,88.095238
AUTO_BACKUP_DISABLED,1,22,4.545455
AUTO_REPAIR_DISABLED,4,16,25.000000
